GradientDescentOptimizer. TensorFlow 2.x

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
!git clone https://github.com/ageron/handson-ml.git   # hounsing data

In [ ]:
housing=pd.read_csv('/content/handson-ml/datasets/housing/housing.csv')

In [ ]:
housing.head()

# understand data

In [ ]:
housing.columns

In [ ]:
housing.info()

In [ ]:
housing.describe()

In [ ]:
housing.corr()

In [ ]:
#matrix correlation 
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
plt.figure(figsize=(20,11))
sns.heatmap(housing.corr(), annot = True, annot_kws={'size':10},linewidths=1,cmap=plt.cm.Reds)

In [ ]:
housing.hist(bins=50, figsize=(20, 15))
plt.show()

In [1]:
#image california map
![](https://th.bing.com/th/id/OIP.QjIh5J-7PpzkcpZ38ncbvgHaG9?w=194&h=184&c=7&r=0&o=5&pid=1.7)

'[]' is not recognized as an internal or external command,
operable program or batch file.
'h' is not recognized as an internal or external command,
operable program or batch file.
'c' is not recognized as an internal or external command,
operable program or batch file.
'r' is not recognized as an internal or external command,
operable program or batch file.
'o' is not recognized as an internal or external command,
operable program or batch file.
'pid' is not recognized as an internal or external command,
operable program or batch file.


In [ ]:
housing.plot(kind='scatter', x='longitude', y='latitude', alpha=0.1)

In [ ]:
housing.plot(kind='scatter', x='longitude', y='latitude', alpha=0.4,s=housing['population']/50,
             label='population', figsize=(12,8), c='median_house_value',cmap=plt.get_cmap('jet'))

In [ ]:
#đếm các giá trị null và không null
print('số các giá trị null:\n ',housing.isnull().sum())
print('--------------------------------------------------')
print('số các giá trị không null:\n ',housing.notnull().sum()) 

In [ ]:
#đếm tỉ lệ isnull trên tổng số liệu trong 1 cột 
total = housing.isnull().sum().sort_values(ascending=False)
count=housing.isnull().count()
percent = (housing.isnull().sum()/housing.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, count], axis=1, keys=['Total', 'count'])
missing_data['percent']=percent
f, ax = plt.subplots(figsize=(15, 6))
#plt.xticks(rotation='90')
sns.barplot(missing_data.index, missing_data['percent'])
plt.xlabel('Features', fontsize=15)
plt.ylabel('Percent of missing values', fontsize=15)
plt.title('Percent missing data by feature', fontsize=15)
missing_data

In [ ]:
import seaborn as sns
%matplotlib inline
plt.figure(figsize=(10,7))
sns.heatmap(housing.isna())

In [ ]:
#số lượng dữ liệu duy nhất
housing.nunique()

In [ ]:
#thống kê định tính
print(housing.iloc[:,-1].describe())

# xử lí miss data

In [ ]:
median = housing["total_bedrooms"].median()
housing["total_bedrooms"].fillna(median, inplace=True)
housing["total_bedrooms"].isnull().sum()

# **combination features**

In [ ]:
housing['room_per_household'] = housing['total_rooms'] / housing['households']

housing['bedroom_per_household'] = housing['total_bedrooms'] / housing['households']

housing['population_per_household'] = housing['population'] / housing['households']

In [ ]:
housing

In [ ]:
housing.corr().median_house_value.sort_values()

# **text to categories**

In [ ]:
housing.ocean_proximity.unique()

In [ ]:
housing_dumies=pd.get_dummies(housing,prefix=['ocean_proximity'])

# **scaling data**

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
 
#call the standard scaler 
std_scaler = StandardScaler()
minmax_scaler=MinMaxScaler()
 
#fit the values to the function 
Stand_Sc = std_scaler.fit_transform(housing_dumies.drop('median_house_value',axis=1))
minmax_sc = minmax_scaler.fit_transform(housing_dumies.drop('median_house_value',axis=1)) 

In [ ]:
#use seaborn for KDE plot
import seaborn as sns
sns.kdeplot(Stand_Sc[:,5],fill=True, color = 'Yellow')

In [ ]:
sns.kdeplot(minmax_sc[:,5],fill=True, color = 'Green')

In [ ]:
x_data=minmax_sc
y_data=minmax_scaler.fit_transform(np.array(housing_dumies['median_house_value']).reshape(x_data.shape[0],1))
print(x_data.shape)
print(y_data.shape)

# **train model**

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x_data,y_data,test_size=0.2)

In [ ]:
n_dim=x_data.shape[1]

In [ ]:
tf.compat.v1.disable_eager_execution()
x=tf.compat.v1.placeholder(tf.float32,[None,n_dim])
y=tf.compat.v1.placeholder(tf.float32,[None,1])
w=tf.Variable(tf.ones([n_dim,1]))
b=tf.Variable(np.random.randn(),dtype=tf.float32)

In [ ]:
pred=tf.add(tf.matmul(x,w),b)

In [ ]:
learning_rate=400
loss=tf.sqrt(tf.reduce_mean(tf.square(y-pred)))
optimizer=tf.compat.v1.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(loss)

In [ ]:
sess=tf.compat.v1.Session()
sess.run(tf.compat.v1.global_variables_initializer())

In [ ]:
epochs=40000
loss_cur=[]
for epoch in range(epochs):
    sess.run(optimizer,feed_dict={x:x_train,y:y_train})
    test_loss=sess.run(loss,feed_dict={x:x_test,y:y_test})
    train_loss=sess.run(loss,feed_dict={x:x_train,y:y_train})
    loss_cur.append([test_loss,train_loss])
    if epoch%400==0:
        print('epoch {} with test loss {} vs train loss {}'.format(epoch,test_loss,train_loss)) 